In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt


# Load a small dataset (example: TensorFlow flowers dataset as placeholder)
# Replace with your recyclable-items dataset
(data_train, labels_train), (data_test, labels_test) = tf.keras.datasets.cifar10.load_data()


# Filter only classes we need (example: use 3 classes)
# In real project, load your own recyclable dataset
classes_to_keep = [0, 1, 2] # 0=paper, 1=plastic, 2=metal (example mapping)
mask_train = np.isin(labels_train, classes_to_keep).flatten()
mask_test = np.isin(labels_test, classes_to_keep).flatten()


data_train = data_train[mask_train] / 255.0
data_test = data_test[mask_test] / 255.0
labels_train = labels_train[mask_train]
labels_test = labels_test[mask_test]


# Simple CNN Model
model = models.Sequential([
layers.Conv2D(16, (3,3), activation='relu', input_shape=(32,32,3)),
layers.MaxPooling2D(2,2),
layers.Conv2D(32, (3,3), activation='relu'),
layers.MaxPooling2D(2,2),
layers.Flatten(),
layers.Dense(64, activation='relu'),
layers.Dense(3, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(data_train, labels_train, epochs=10, validation_split=0.2)


model.save("recycle_model.h5")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 315s 2us/step


C:\Users\PC\anaconda3\New folder\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.7322 - loss: 0.6450 - val_accuracy: 0.8083 - val_loss: 0.4889
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.8134 - loss: 0.4668 - val_accuracy: 0.8173 - val_loss: 0.4625
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8420 - loss: 0.4021 - val_accuracy: 0.8293 - val_loss: 0.4389
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8552 - loss: 0.3700 - val_accuracy: 0.8387 - val_loss: 0.4283
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8684 - loss: 0.3442 - val_accuracy: 0.8517 - val_loss: 0.3897
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8748 - loss: 0.3216 - val_accuracy: 0.8660 - val_loss: 0.3692
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8813 - loss: 0.3039 - val_accuracy: 0.8653 - val_loss: 0.3577
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.8883 - loss: 0.2828 - val_acc

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open("recycle_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\PC\AppData\Local\Temp\tmplvo_eics\assets


INFO:tensorflow:Assets written to: C:\Users\PC\AppData\Local\Temp\tmplvo_eics\assets


Saved artifact at 'C:\Users\PC\AppData\Local\Temp\tmplvo_eics'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1287337586000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337587536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337586768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337587344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337587728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337588112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337587920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1287337588496: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [4]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="recycle_model.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# Pick one test image
test_img = data_test[0:1]


interpreter.set_tensor(input_details[0]['index'], test_img.astype(np.float32))
interpreter.invoke()


pred = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(pred)
print("Predicted class:", predicted_class)

C:\Users\PC\anaconda3\New folder\anaconda\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Predicted class: 0
